In [ ]:
from nba_api.stats.endpoints import ShotChartDetail, playercareerstats
from nba_api.stats.static import players
from time import sleep
import pandas as pd
import matplotlib.pyplot as plt
import os
import utils.nba_plot_utils as pu
import importlib
from tqdm import tqdm
%matplotlib inline


In [23]:
# all of Stephen Curry's shots in the 2020-21 season
player_id = 201939  # Stephen Curry
season = '2020-21'

In [24]:
DATA_FOLDER = 'data/'

In [25]:
# pull the shot chart data
response = ShotChartDetail(
    player_id=player_id,
    team_id=0,  # 0 indicates that we want data for all teams
    season_nullable=season,
    context_measure_simple="FGA"  # FGA indicates that we want data on field goal attempts
)

In [26]:

# convert to a DataFrame
shot_data = response.get_data_frames()[0]

In [27]:
# save as a file
file_path = os.path.join(DATA_FOLDER, f'steph_curry_full_shots_{season}.csv')

In [28]:
shot_data.to_csv(file_path, index=False)

In [ ]:
print(f"Data successfully downloaded and saved as '{file_path}'")
print(shot_data['SHOT_MADE_FLAG'].unique())

In [30]:
# example - steph curry shot chart and visualization
# shot_data = pd.read_csv(file_path)

In [31]:
# drop rows where the shot distance is missing
shot_data = shot_data.dropna(subset=['LOC_X', 'LOC_Y'])

In [ ]:
importlib.reload(pu)

In [ ]:
shot_data.LOC_X.max()

In [ ]:
fin, finax = pu.plot_shot_scatter(shot_data, plot_title=f'Stephen Curry Shot Chart {season}')

In [ ]:
def get_season_players(all_activities, season):
    return all_activities[all_activities['SEASON_ID'] == season].PLYER_ID.unique().tolist()

In [ ]:
def get_active_players_from_period(start_year, end_year):
    period_players = set()
    for year in range(start_year, end_year + 1):
        season = f"{year}-{str(year + 1)[2:]}"
        print(f"Getting players for {season}")
        players.add(get_season_players)
        sleep(1)
        
    return period_players

In [149]:
def scrape_all_player_active_seasons():

    all_players = [p['id'] for p in players.get_players()]
    print(all_players)

    all_activities = []
    for player in tqdm(all_players, desc='Scraping player activity'):
        # Fetch career stats for each player
        player_stats = playercareerstats.PlayerCareerStats(player_id=player)
        
        # Get the player's data as a DataFrame
        player_data = player_stats.get_data_frames()[0]
        player_active_years = player_data[['PLAYER_ID', 'SEASON_ID']]

        all_activities.append(player_active_years)
        sleep(0.2)
    all_activities = pd.concat(all_activities, ignore_index=True)  
    return all_activities  

In [ ]:
all_activities = scrape_all_player_active_seasons()

In [ ]:
# get all players that played in the 2020-21 season
active_season_players = get_season_players(start_year=2020, end_year=2021)
print(f"Total players fetched: {len(all_players)}")

In [101]:
def get_shot_data(player_id, season):
    try:
        # שליפת נתוני זריקות לשחקן מסוים
        response = ShotChartDetail(
            player_id=player_id,
            team_id=0,  # כל הקבוצות
            season_nullable=season,
            context_measure_simple="FGA"  # כל הזריקות (כולל החטאות)
        )
        data = response.get_data_frames()[0]
        data['PLAYER_ID'] = player_id
        return data
    
    except Exception as e:
        print(f"Error fetching data for player {player_id}: {e}")
        return None

In [152]:
# gather data for all players
all_shot_data = []
#period_active_players = get_active_players_from_period(2020, 2021)

for player in all_players:
    player_id = player['id']
    player_name = player['full_name']
    print(f"Fetching shot data for {player_name} (ID: {player_id})...")
    
    shot_data = get_shot_data(player_id, season)
    if shot_data is not None and not shot_data.empty:
        all_shot_data.append(shot_data)
    sleep(0.3)  # time delay to avoid overwhelming the API

# concat and save to csv
if all_shot_data:
    final_data = pd.concat(all_shot_data, ignore_index=True)
    output_file = os.path.join(DATA_FOLDER, f'nba_shots_{season}_all_players.csv')
    final_data.to_csv(output_file, index=False)
    
    print("Data successfully saved to 'nba_shots_2020-21_all_players.csv'")
else:
    print("No shot data was fetched.")

Fetching shot data for Allan Bristow (ID: 76257)...
Fetching shot data for Tyrone Britt (ID: 76258)...
Fetching shot data for Wayman Britt (ID: 76259)...
Fetching shot data for Mike Brittain (ID: 76260)...
Fetching shot data for David Britton (ID: 76261)...
Fetching shot data for Izaiah Brockington (ID: 1631167)...
Fetching shot data for Jon Brockman (ID: 201972)...
Fetching shot data for Ryan Broekhoff (ID: 1629151)...
Fetching shot data for Jim Brogan (ID: 76262)...
Fetching shot data for Malcolm Brogdon (ID: 1627763)...
Fetching shot data for Gary Brokaw (ID: 76263)...
Fetching shot data for Price Brookfield (ID: 76264)...
Fetching shot data for Aaron Brooks (ID: 201166)...
Fetching shot data for Armoni Brooks (ID: 1629717)...
Fetching shot data for Dillon Brooks (ID: 1628415)...
Fetching shot data for Kevin Brooks (ID: 76265)...
Fetching shot data for MarShon Brooks (ID: 202705)...
Fetching shot data for Michael Brooks (ID: 76266)...
Fetching shot data for Scott Brooks (ID: 418)...

In [153]:
from nba_api.stats.endpoints import ShotChartDetail
import pandas as pd
#נתנוי הזריקות של סטף קרי לאורך כל הקריירה 
#כמו הקובץ הקודם אשמח שתריץ אצלך אין לי וויפי בבסיס

# מזהה השחקן של סטף קרי
player_id = 201939

# משיכת נתוני הזריקות של סטף קרי לאורך כל הקריירה
response = ShotChartDetail(
    player_id=player_id,
    team_id=0,  # 0 מכוון לכל הקריירה ולא לקבוצה מסוימת
    season_nullable='ALL',  # כל העונות
    context_measure_simple='FGA'  # נתוני כל הזריקות
)

# קבלת הנתונים בפורמט JSON
data = response.get_json()

# יצירת DataFrame מתוך הנתונים
shot_data = response.shot_chart_detail.get_data_frame()

# שמירת הנתונים לקובץ CSV
output_file = 'stephen_curry_shot_data.csv'
shot_data.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"נתוני הזריקות של סטף קרי נשמרו בקובץ {output_file}")


נתוני הזריקות של סטף קרי נשמרו בקובץ stephen_curry_shot_data.csv


In [ ]:
from nba_api.stats.endpoints import ShotChartDetail
import pandas as pd
#נתוני הזריקות של סטף בעונת 23-24
# מזהה השחקן של סטף קרי
player_id = 201939

# ננסה לשלוף נתוני זריקות עבור עונת 2023-2024
response = ShotChartDetail(
    player_id=player_id,
    team_id=0,
    season_nullable='2023-24',  # עונה ספציפית
    context_measure_simple='FGA'
)

# יצירת DataFrame מתוך הנתונים
shot_data = response.shot_chart_detail.get_data_frame()

# בדיקת הנתונים
if shot_data.empty:
    print("לא נמצאו נתונים גם לעונה הספציפית.")
else:
    print("נתונים נשלפו בהצלחה לעונה 2023-24.")
    print(shot_data.head())

    # שמירת הנתונים לקובץ CSV
    output_file = 'stephen_curry_shot_data_2023_24.csv'
    shot_data.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"נתוני הזריקות נשמרו בקובץ {output_file}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#ויזואליזציה של סטף 23-24
# קריאת קובץ הנתונים של 2020-21
file_path = "/Users/home/stephen_curry_shot_data_2023_24.csv"  # שים את הקובץ הנכון פה
data = pd.read_csv(file_path)

# פילטר לזריקות בלבד (לא ריקות)
data = data.dropna(subset=['LOC_X', 'LOC_Y'])

# פונקציה לציור מגרש כדורסל
def draw_court(ax=None):
    if ax is None:
        ax = plt.gca()
    # גבולות המגרש
    hoop = plt.Circle((0, 0), 7.5, linewidth=2, color="black", fill=False)
    backboard = plt.Rectangle((-30, -7.5), 60, -1, linewidth=2, color="black")
    outer_box = plt.Rectangle((-80, -47.5), 160, 190, linewidth=2, color="black", fill=False)
    free_throw = plt.Circle((0, 142.5), 60, linewidth=2, color="black", fill=False)
    restricted = plt.Circle((0, 0), 40, linewidth=2, color="black", fill=False)

    # הוספת אלמנטים למגרש
    ax.add_patch(hoop)
    ax.add_patch(backboard)
    ax.add_patch(outer_box)
    ax.add_patch(free_throw)
    ax.add_patch(restricted)

    # קווי המגרש
    ax.plot([-220, 220], [-47.5, -47.5], linewidth=2, color="black")  # קו תחתון
    ax.plot([-220, -220], [-47.5, 422.5], linewidth=2, color="black")  # קו שמאלי
    ax.plot([220, 220], [-47.5, 422.5], linewidth=2, color="black")  # קו ימני

    # הגדרת צירים
    ax.set_xlim(-250, 250)
    ax.set_ylim(-50, 470)
    ax.set_aspect('equal')
    ax.axis('off')

# יצירת גרף עם מפת זריקות
%matplotlib inline
plt.figure(figsize=(12, 11))
ax = plt.gca()
draw_court(ax)

# ציור הזריקות: ירוק להצלחות, אדום להחטאות
made_shots = data[data['SHOT_MADE_FLAG'] == 1]
missed_shots = data[data['SHOT_MADE_FLAG'] == 0]

plt.scatter(made_shots['LOC_X'], made_shots['LOC_Y'], c='green', label="Made", s=10)
plt.scatter(missed_shots['LOC_X'], missed_shots['LOC_Y'], c='red', label="Missed", s=10)

# כותרת ופרטים
plt.title("NBA Shot Chart - 2020-21 Season", fontsize=15)
plt.legend(loc='upper right')
plt.show()


In [ ]:
from nba_api.stats.endpoints import ShotChartDetail
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# פונקציה לשליפת נתוני זריקות עבור עונה נתונה
def fetch_shot_data(player_id, season):
    response = ShotChartDetail(
        player_id=player_id,
        team_id=0,
        season_nullable=season,
        context_measure_simple='FGA'
    )
    return response.shot_chart_detail.get_data_frame()

# מזהה השחקן של סטף קרי
player_id = 201939

# שליפת נתוני זריקות לשתי העונות
season_2023_24 = fetch_shot_data(player_id, '2023-24')
season_2020_21 = fetch_shot_data(player_id, '2020-21')

# הוספת עמודת עונה
season_2023_24['Season'] = '2023-24'
season_2020_21['Season'] = '2020-21'

# איחוד הנתונים לשתי העונות
combined_data = pd.concat([season_2023_24, season_2020_21])

# יצירת ויזואליזציה
plt.figure(figsize=(12, 8))
sns.scatterplot(
    data=combined_data,
    x='LOC_X', y='LOC_Y',
    hue='Season',
    style='SHOT_MADE_FLAG',  # סמל שונה לפי קליעה או החטאה
    palette='coolwarm',
    alpha=0.7
)
plt.title('Steph Curry Shot Chart Comparison: 2023-24 vs 2020-21', fontsize=16)
plt.xlabel('Court X Location')
plt.ylabel('Court Y Location')
plt.legend(title='Season')
plt.axhline(0, color='black', linewidth=1)  # קו לציון הקשת
plt.grid(alpha=0.3)

# שמירת הגרף לקובץ
chart_filename = 'steph_curry_shot_chart_comparison.png'
plt.savefig(chart_filename, dpi=300, bbox_inches='tight')  # שמירת קובץ עם איכות גבוהה
print(f"הגרף נשמר בהצלחה בקובץ {chart_filename}")

# הצגת הגרף
plt.show()
